In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt


In [13]:
df = pd.read_csv('voting_locations.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')
gdf_final = gdf[['name', 'street', 'type', 'language','geometry']]
gdf_final['language'] = np.where(gdf_final['language'].isna(), 'English only', gdf_final['language'])


# add city boundary
city_boundary = pd.read_csv('city_boundary.csv')
city_boundary['the_geom'] = city_boundary['the_geom'].apply(wkt.loads)
city_boundary = gpd.GeoDataFrame(city_boundary, geometry='the_geom', crs='EPSG:4326')
city_boundary['type'] = 'bounds' 
city_boundary['name'] = 'Chicago'
city_boundary.rename(columns={'the_geom': 'geometry'}, inplace=True)
city_boundary = city_boundary[['name','type', 'geometry']]

# add hyde park boundary 
ca_boundaries = gpd.read_file('comm_areas.geojson')
ca_boundaries = ca_boundaries.to_crs(epsg=4326)
ca_boundaries = ca_boundaries[['community', 'geometry']]
ca_boundaries['community'] = ca_boundaries['community'].str.title()
ca_boundaries.rename(columns={'community': 'name'}, inplace=True)
ca_boundaries['type'] = 'bounds'

# add center of mass for each community area
ca_boundaries = ca_boundaries.to_crs("EPSG:26971")
ca_boundaries["centroid"] = ca_boundaries["geometry"].centroid
# convert centriod to lat lon
ca_boundaries["centroid"] = ca_boundaries["centroid"].to_crs("EPSG:4326")
ca_boundaries["center_lon"] = ca_boundaries["centroid"].x
ca_boundaries["center_lat"] = ca_boundaries["centroid"].y
ca_boundaries.drop(columns=["centroid"], inplace=True)
ca_boundaries["geometry"] = ca_boundaries["geometry"].to_crs("EPSG:4326")



# add hp to gdf as a new row
gdf_final = pd.concat([gdf_final, ca_boundaries, city_boundary], ignore_index=True)
gdf_final.to_file('voting_locations.geojson', driver='GeoJSON')


/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
gdf_final

,name,street,type,language,geometry,center_lon,center_lat
0,Goldblatts Building,1615 W. Chicago Ave.,Normal,Spanish and Polish,POINT (-87.66775 41.89591),NaN,NaN
1,Near North Library,310 W. Division St.,Normal,English only,POINT (-87.63665 41.90412),NaN,NaN
2,Dawson Technical Institute,3901 S. State St.,Normal,English only,POINT (-87.62601 41.82281),NaN,NaN
3,Dr. Martin Luther King Center,4314 S. Cottage Gr.,Normal,English only,POINT (-87.60708 41.81620),NaN,NaN
4,Southside YMCA,6330 S. Stony Island Ave.,Normal,English only,POINT (-87.58739 41.77945),NaN,NaN
...,...,...,...,...,...,...,...
128,Morgan Park,NaN,bounds,NaN,"MULTIPOLYGON (((-87.64215 41.68508, -87.64249 ...",-87.669054,41.689730
129,Ohare,NaN,bounds,NaN,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ...",-87.893694,41.975684
130,Edgewater,NaN,bounds,NaN,"MULTIPOLYGON (((-87.65456 41.99817, -87.65456 ...",-87.663416,41.986712
131,Edison Park,NaN,bounds,NaN,"MULTIPOLYGON (((-87.80676 42.00084, -87.80676 ...",-87.813781,42.007613
